In [23]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def load_image(image_path):
    image = Image.open(image_path)
    image = image.resize((160, 160))  # Resize to the input size expected by FaceNet
    image = np.asarray(image)
    image = image.astype('float32')
    mean, std = image.mean(), image.std()
    image = (image - mean) / std
    image = np.expand_dims(image, axis=0)
    return image

def load_pb_model(model_filepath):
    # Load TensorFlow Graph from .pb file
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()

    with open(model_filepath, "rb") as f:
        graph_def.ParseFromString(f.read())
    
    with graph.as_default():
        tf.import_graph_def(graph_def, name="")

    return graph

def generate_embeddings(graph, dataset_path):
    embeddings = []
    labels = []
    with tf.compat.v1.Session(graph=graph) as sess:
        for person_name in os.listdir(dataset_path):
            person_path = os.path.join(dataset_path, person_name)
            if os.path.isdir(person_path):  # Add this check here
                for image_name in os.listdir(person_path):
                    image_path = os.path.join(person_path, image_name)
                    if os.path.isfile(image_path):  # Optionally, ensure this is a file
                        image = load_image(image_path)
                        
                        # Fetch the tensors
                        images_placeholder = graph.get_tensor_by_name("input:0")
                        embeddings_tensor = graph.get_tensor_by_name("embeddings:0")
                        phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")

                        feed_dict = {images_placeholder: image, phase_train_placeholder: False}
                        embedding = sess.run(embeddings_tensor, feed_dict=feed_dict)
                        
                        embeddings.append(embedding)
                        labels.append(person_name)
    embeddings = np.asarray(embeddings).reshape(len(embeddings), -1)  # Reshape to (num_samples, embedding_size)
    labels = np.asarray(labels)
    return embeddings, labels


model_filepath = '20180402-114759/20180402-114759.pb'  # Update this path to your .pb model file
graph = load_pb_model(model_filepath)
dataset_path = 'dataset'  # Update this path to your dataset
embeddings, labels = generate_embeddings(graph, dataset_path)


In [24]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from tqdm import tqdm

def preprocess_image(image_path, target_size=(160, 160)):
    """Load an image file, resize it to target size, and normalize it."""
    image = Image.open(image_path)
    image = image.resize(target_size)
    image_array = np.array(image)
    # Normalize the image
    image_array = image_array.astype('float32')
    mean, std = image_array.mean(), image_array.std()
    image_array = (image_array - mean) / std
    return image_array

def load_dataset(dataset_path, target_size=(160, 160)):
    """Load and preprocess the entire dataset."""
    X, y = [], []
    # Walk through the person directories and list their images
    for person_name in tqdm(os.listdir(dataset_path), desc="Processing"):
        person_path = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_path):
            continue  # Skip non-directory files
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            if not os.path.isfile(image_path) or not image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue  # Skip non-image files
            image_array = preprocess_image(image_path, target_size)
            X.append(image_array)
            y.append(person_name)
    return np.array(X), np.array(y)

# Path to your dataset
dataset_path = 'dataset'

# Load and preprocess the dataset
X, y = load_dataset(dataset_path)

print(f"Loaded and preprocessed {len(X)} images.")


Processing: 100%|██████████| 5750/5750 [00:26<00:00, 219.60it/s]


Loaded and preprocessed 13233 images.


In [21]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam


# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_encoded, test_size=0.2, random_state=42)

# Define the new model architecture
embedding_input_size = embeddings.shape[1]  # This should match the size of your FaceNet embeddings
input_layer = Input(shape=(embedding_input_size,))
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.5)(x)
output_layer = Dense(len(np.unique(labels_encoded)), activation='softmax')(x)

# Define the model
classifier_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
classifier_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = classifier_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)



Epoch 1/10
1/1 [==============================] - 0s 284ms/step - loss: 1.3999 - accuracy: 0.0000e+00 - val_loss: 1.3074 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 19ms/step - loss: 1.3325 - accuracy: 0.6667 - val_loss: 1.3129 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 20ms/step - loss: 1.3047 - accuracy: 0.6667 - val_loss: 1.3178 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 20ms/step - loss: 1.2689 - accuracy: 0.6667 - val_loss: 1.3207 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 22ms/step - loss: 1.1867 - accuracy: 1.0000 - val_loss: 1.3235 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 20ms/step - loss: 1.1806 - accuracy: 1.0000 - val_loss: 1.3265 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 20ms/step - loss: 1.1034 - accuracy: 1.0000 - val_loss: 1.3299 - val_accuracy: 1.0000
Epoch 8/10
1/1 [======

In [22]:
# Evaluate the model on the test set
test_loss, test_accuracy = classifier_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


1/1 [==============================] - 0s 18ms/step - loss: 1.3420 - accuracy: 0.0000e+00
Test Accuracy: 0.00%


In [ ]:
# Save the model
model_save_path = 'path_to_save_your_model/facenet_classifier.h5'
classifier_model.save(model_save_path)

# To save just the weights
weights_save_path = 'path_to_save_your_model/facenet_classifier_weights.h5'
classifier_model.save_weights(weights_save_path)


In [ ]:
def predict_person(image_path, facenet_model, classifier_model, label_encoder):
    # Load and preprocess the image
    image = load_image(image_path)  # This should be the same function you used before

    # Generate embedding using FaceNet
    with tf.compat.v1.Session(graph=facenet_model) as sess:
        images_placeholder = facenet_model.get_tensor_by_name("input:0")
        embeddings_tensor = facenet_model.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = facenet_model.get_tensor_by_name("phase_train:0")
        
        feed_dict = {images_placeholder: image, phase_train_placeholder: False}
        embedding = sess.run(embeddings_tensor, feed_dict=feed_dict)

    embedding = np.reshape(embedding, (1, -1))  # Reshape the embedding to match the classifier's input expectation
    
    # Predict using the classifier model
    prediction = classifier_model.predict(embedding)
    predicted_label_idx = np.argmax(prediction, axis=1)
    predicted_label = label_encoder.inverse_transform(predicted_label_idx)
    
    return predicted_label

# Example usage
image_path = 'path_to_your_new_image.jpg'
facenet_graph = load_pb_model('20180402-114759/20180402-114759.pb')  # Load the FaceNet model as before
predicted_person = predict_person(image_path, facenet_graph, classifier_model, label_encoder)
print(f"Predicted person: {predicted_person}")
